<a href="https://colab.research.google.com/github/tonystz/gitpod/blob/main/pycuda_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CUDA and PyCUDA

In [1]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 12.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp39-cp39-linux_x86_64.whl size=661964 sha256=3c7a8bb93c78ea9bb646a0e7df8cf3d8d84bf74dbbe4e2a848e21941713caab3
  Stored in directory: /root/.cache/pip/wheels/82/09/27/74d8e31ed19c530166e0d263aabe1ea57465e255615bda8fc0
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69870 sha256=f3e33f3871ffb977c0fe80f0b95c1bee797ac68d123192e1d91e1a28d7ae2bf5
  Stored in directory: /root/.cache/pip/wheels/76/ee/

In [ ]:
import numpy
numpy.random.seed(1729)
a = numpy.random.randn(4,4)

In [ ]:
a = a.astype(numpy.float32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [ ]:
cuda.memcpy_htod(a_gpu, a)


In [ ]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [ ]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [ ]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

In [ ]:
b = numpy.random.randn(4,4)
b = b.astype(numpy.float32)
c = numpy.random.randn(4,4)
c = c.astype(numpy.float32)

In [ ]:
mod2 = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

In [ ]:
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)


In [ ]:
func = mod2.get_function("add2")
func(b_gpu,c_gpu, block=(4,4,1))

In [ ]:
added = numpy.empty_like(b)
cuda.memcpy_dtoh(added, b_gpu)
print(added)
print(b)
print(c)

# Exercises

1. Write a cuda kernel to find the elementwise square of a matrix
2. Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero
3. Write a cuda kernel to multiply two matrices:
    1. Assume square matrices, with dimensions < 1024
    2. Assume square matrices, with dimensions > 1024
    3. Assume non-square matrices, with dimensions > 1024

In [ ]:
1. 